In [1]:
#Import libraries
import cv2
import mediapipe as mp

In [2]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Finger tip and pip landmark indices
finger_tips = [4, 8, 12, 16, 20]
finger_pips = [3, 6, 10, 14, 18]

In [3]:
# Start video capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        break

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks, hand_info in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_label = hand_info.classification[0].label  # 'Left' or 'Right'
            fingers_up = []

            # Thumb logic
            if hand_label == "Right":
                if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
                    fingers_up.append(1)
            else:  # Left hand
                if hand_landmarks.landmark[4].x > hand_landmarks.landmark[3].x:
                    fingers_up.append(1)

            # Other fingers
            for i in range(1, 5):
                tip_id = finger_tips[i]
                pip_id = finger_pips[i]
                if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[pip_id].y:
                    fingers_up.append(i + 1)

            # Draw landmarks
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Display finger numbers
            cv2.putText(img, f"Fingers Up: {fingers_up}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Finger Counter", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()